# Inverse kinematics of a 3dof manipulator
Consider the following manipulator

<img src="https://github.com/robotica-cem/cinematica-notebooks/blob/main/figures/3d-3dof-revolute.png?raw=true" width=500 />

The inverse kinematics can be stated as follows. Given a desired position $p$ of the tool center point at the end of the second link, determine the joint angles $\theta_1$, $\theta_2$ and $\theta_3$ that achieves this position.

There are two steps to the solution:
1. Determine the first joint angle $\theta_1$. The rotation about the first, vertical axis will determine the vertical plane in which the manipulator arm will move. All rotations in the subsequent joints will just move the arm in this vertical plane. Thus, we can find the angle $\theta_1$ by considering the $x$ and $y$ coordinates of the point $p$.
$$ \theta_1 = atan2(p_y, p_x), $$
where the $atan2$ function is the inverste tangent function that keeps track of the quadrant of the angle.
2. In the vertical plane in which the arm is now constrained, the inverse kinematics problem is a geometric problem in the plane. Consider the below figure.
<img src="https://github.com/robotica-cem/cinematica-notebooks/blob/main/figures/2d-RR-inverse-kinematics.png?raw=true" width=500 />
Using the law of cosines 
$$ d_0^2 = l_1^2 + l_2^2 -2l_1l_2\cos\alpha,$$
where $d_0 = \sqrt{p_x^2 + p_y^2}$,
we obtain
$$ \cos\alpha = \frac{l_1^2 + l_2^2 - d_0^2}{2l_1l_2} = D_\alpha. $$
Since $\sin^2\alpha = 1 - \cos^2\alpha = 1 - D_\alpha^2$
$$ \tan\alpha = \frac{\sin\alpha}{\cos\alpha}  = \frac{\pm \sqrt{1- D_\alpha^2}}{D_\alpha} \quad \Rightarrow \quad \alpha = atan2(\pm \sqrt{1-D_\alpha^2}, D_\alpha).$$
And so,
$$ \theta_3 = \frac{\pi}{2} - \alpha. $$
Using again, the law of cosines
$$ l_2^2 = l_1^2 + d_0^2 - 2l_1d_0\cos\beta$$
we get
$$ \cos\beta = \frac{l_1^2 + d_0^2 - l_2^2}{2l_1d_0} = D_\beta, \; \text{and} $$
$$ \beta = atan2(\sqrt{1-D_\beta^2}, D_\beta). $$
The angle $\psi$ can be found from the inverse tangent as 
$$ \psi = atan2(p_z, d_1), $$ 
where $d_1 = \sqrt{p_x^2 + p_y^2}$.

Finally, the second angle becomes
$$\theta_2 = \frac{\pi}{2} - \psi -\beta. $$
This corresponds to the "elbow up" solution. There is also a possible solution with the elbow down. 
$$\theta_2 = \frac{\pi}{2} - \psi + \beta, \qquad \theta_3 = \frac{\pi}{2} + \alpha.$$ 

In [1]:
import numpy as np
import doctest

In [28]:
def fwd_kinematics_RRR(th1, th2, th3, l1=2, l2=1):
    '''
    Implements the forward kinematics of a robot with two perpendicular
    revolute joints connecting the first link to the base, and one revolute joint
    connecting link one and two. In its default position (for all joint angles 
    equal to zero), the first link is aligned with the global z-axis (vertical) and the 
    second link is in the direction of the positive global x-axis.
    
    Arguments
    ---------
    th1, th2, th3 : float
       Angle in radians of the three degree of freedoms, respectively.
    l1, l2 : float
       Length of the two links, respectively.
 
    Returns
    -------
    x : float
       The position in the global x-direction of the end-effector (tool point)
    y : float
       The position in the global y-direction of the end-effector (tool point)
    z : float
       The position in the global z-direction of the end-effector (tool point)
    j : tuple with 3 elements
       The position of the joint between the two links
    
    Tests
    ------
    1) End-effector pose at default position
    >>> x, y, z, j = fwd_kinematics_RRR(0, 0, 0)
    >>> "(%0.2f, %0.2f, %0.2f)" %(x, y, z)
    '(1.00, 0.00, 2.00)'
    
    2) End-effector pose at 90 degrees in all three joints
    >>> x, y, z, j = fwd_kinematics_RRR(np.pi/2, np.pi/2, np.pi/2)
    >>> "(%0.2f, %0.2f, %0.2f)" %(x, y, z)
    '(0.00, 1.00, 0.00)'
    
    3) End-effector pose at 0 degress in first joint and 90 degress in second
    >>> x, y, z, j = fwd_kinematics_RRR(0, np.pi/2, 0)
    >>> "(%0.2f, %0.2f, %0.2f)" %(x, y, z)
    '(2.00, 0.00, -1.00)'
    
    4) End-effector position is always inside a sphere of a certain radius
    >>> poses = [fwd_kinematics_RRR(q1_, q2_, q3_)
    ... for q1_ in np.arange(0, 2*np.pi, 0.4) 
    ... for q2_ in np.arange(-np.pi, np.pi, 0.4) 
    ... for q3_ in np.arange(0, np.pi/2, 0.4)]
    >>> distances = np.array([np.sqrt(x_**2 + y_**2 + z_**2) for x_, y_, z_, j_ in poses])
    >>> max_radius = 3 + 1e-12 # Add a small tolerance
    >>> np.any(distances > max_radius)
    False
    
    5) Joint is always at constant distance from the origin
    >>> poses = [fwd_kinematics_RRR(q1_, q2_, 0)
    ... for q1_ in np.arange(0, 2*np.pi, 0.4) 
    ... for q2_ in np.arange(-np.pi, np.pi, 0.4)]
    >>> distances = np.array([np.sqrt(j_[0]**2 + j_[1]**2 + j_[2]**2) 
    ... for x_, y_, z_, j_ in poses])
    >>> np.any(np.abs(distances-2) > 1e-12)
    False
    
    '''

    # For convenience
    c1 = np.cos(th1)
    s1 = np.sin(th1)
    c2 = np.cos(th2)
    s2 = np.sin(th2)
    c3 = np.cos(th3)
    s3 = np.sin(th3)
    
    origin = np.array([0,0,0])
    zer = np.array([0,0,0])
    
    # First joint, rotation about z.
    T_01 = np.eye(4);
    T_01[:3, 0] = [c1, s1, 0] # x_1 in {0}
    T_01[:3, 1] = [-s1, c1, 0] # y_1 in {0}
    
    # Second joint, rotation about y
    T_12 = np.eye(4);
    T_12[:3, 0] = [c2, 0, -s2] # x_2 in {1}
    T_12[:3, 2] = [s2, 0, c2] # z_2 in {1}
    
    # Link 1, origin in joint 3, directions same as {2}
    T_2L1 = np.eye(4)
    T_2L1[:3, 3] = [0, 0, l1]
    
    # Third joint, connecting the two segments. Rotation about y_L1 = y_2 = y1
    T_L13 = np.eye(4)
    T_L13[:3, 0] = [c3, 0, -s3] # x_2 in {1}
    T_L13[:3, 2] = [s3, 0, c3] # z_2 in {1}
    
    # Second link, origin at tip, directions same as {3}.
    T_3E = np.eye(4)
    T_3E[:3, 3] = [l2, 0, 0]
    
    # Combined transform
    T_0E = T_01 @ T_12 @ T_2L1 @ T_L13 @ T_3E
    
    # Tool tip centre, origin of E
    tcp0 = np.array([0, 0, 0, 1])
    tcp = T_0E @ tcp0
    
    # Joint 3's position
    T_0L1 = T_01 @ T_12 @ T_2L1
    j = T_0L1[:3,3]
    
    return (*tcp[:3], j)

def inverse_kinematics_RRR(tcp, l1=2, l2=1):
    '''
    Inverse kinematics of the 3dof manipulator above.
    
    Arguments
    ---------
    tcp : array-like of length 3
       Tool center point, i.e. the desired position of the end-effector.
    l1, l2 : float
       Length of the two links, respectively.
       
    Returns
    -------
    th1, th2, th3 : float
       The joint angles in radians.
       
    Tests
    -----
    1) Arm stretched out along the x-axis
    >>> tcp = [3,0,0]
    >>> th1, th2, th3 = inverse_kinematics_RRR(tcp)
    >>> np.allclose([th1, th2, th3], [0, np.pi/2, -np.pi/2])
    True
    
    2) Arm just rotated 90 degrees about first joint
    >>> tcp = [0,1,2]
    >>> th1, th2, th3 = inverse_kinematics_RRR(tcp)
    >>> np.allclose([th1, th2, th3], [np.pi/2, 0, 0])
    True
    
    3) Use the forward map for some random angles. Make sure the elbow up
    configuration is used.
    >>> ths = np.random.randn(3)
    >>> th1 = ths[0]
    >>> th2 = np.clip(ths[1], 0, np.pi)
    >>> th3 = np.clip(ths[2], -np.pi/2, np.pi/2)
    >>> (x, y, z, j) = fwd_kinematics_RRR(th1, th2, th3)
    >>> th1i, th2i, th3i = inverse_kinematics_RRR([x,y,z])
    >>> np.allclose( [th1,th2,th3], [th1i, th2i, th3i])
    True
    '''
    
    th1 = 0
    th2 = 0
    th3 = 0
    
    return (th1, th2, th3)

## Run doctests
If tests pass, no output is generated.

In [29]:
doctest.run_docstring_examples(fwd_kinematics_RRR, globals())
doctest.run_docstring_examples(inverse_kinematics_RRR, globals())